In [1]:
import pandas as pd
import numpy as np

# Merging DataFrames

In [4]:
list('asdlfkjasdf')

['a', 's', 'd', 'l', 'f', 'k', 'j', 'a', 's', 'd', 'f']

In [5]:
df1 = pd.DataFrame({'key': list('bbacaab'),
                   'data1': range(7)})

In [7]:
df2 = pd.DataFrame({'key': list('abd'),
                   'data2': range(3)})

In [8]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [9]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [10]:
pd.merge(df1,df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


By default, Pandas does an inner join on the valus of the shared columns

In [14]:
df3 = pd.DataFrame({'key': list('kjl'),
                   'data2': range(3)})

In [15]:
pd.merge(df1,df3)

,data1,key,data2


In [16]:
df4 = pd.DataFrame({'lkey': list('bbacaab'),
                   'data1': range(7)})
df5 = pd.DataFrame({'rkey': list('abd'),
                   'data2': range(3)})

In [22]:
pd.merge(df4, df5)

MergeError: No common columns to perform merge on

In [21]:
pd.merge(df4, df5, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


If we want to use columns that do not have the same name in both dataframes and/or do something other than an inner join, we need to specify

In [25]:
pd.merge(df4, df5, left_on='lkey', right_on='rkey', how='outer')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a
6,3,c,NaN,NaN
7,NaN,NaN,2,d


In [30]:
left = pd.DataFrame({
        'key1': list('aab'),
        'key2': list('bnm'),
        'lval': range(3)
    })

right = pd.DataFrame({
        'key1': list('aabb'),
        'key2': list('bbbm'),
        'rval': [6,7,8,9]
    })

In [31]:
right

,key1,key2,rval
0,a,b,6
1,a,b,7
2,b,b,8
3,b,m,9


In [29]:
left

,key1,key2,lval
0,a,b,0
1,a,n,1
2,b,m,2


We can do merges on more than one column

In [32]:
pd.merge(left, right, on=['key1','key2'])

,key1,key2,lval,rval
0,a,b,0,6
1,a,b,0,7
2,b,m,2,9


Or keep the extra columns in the resulting dataframe, adding suffixes that we can specify.

In [33]:
pd.merge(left, right, on=['key1'])

,key1,key2_x,lval,key2_y,rval
0,a,b,0,b,6
1,a,b,0,b,7
2,a,n,1,b,6
3,a,n,1,b,7
4,b,m,2,b,8
5,b,m,2,m,9


In [34]:
pd.merge(left, right, on=['key1'], suffixes=['_left', '_right'])

,key1,key2_left,lval,key2_right,rval
0,a,b,0,b,6
1,a,b,0,b,7
2,a,n,1,b,6
3,a,n,1,b,7
4,b,m,2,b,8
5,b,m,2,m,9


## Merging on index
We can also use the index of one or the two dataframes as merging keys

In [36]:
left1 = left

In [37]:
right1 = pd.DataFrame({'vals': [5,8]}, index=['a','b'])

In [38]:
right1

,vals
a,5
b,8


In [39]:
pd.merge(left1, right1, left_on='key1', right_index=True)

,key1,key2,lval,vals
0,a,b,0,5
1,a,n,1,5
2,b,m,2,8


In [40]:
left

,key1,key2,lval
0,a,b,0
1,a,n,1
2,b,m,2


## Hierarchical indexing
is the use of a multi-level index.

In [43]:
right_h = pd.DataFrame(np.arange(12).reshape(6,2),
                      index = [list('aabbxx'),list('bbnnnn')]
                      )

In [44]:
right_h

0   1
a b   0   1
  b   2   3
b n   4   5
  n   6   7
x n   8   9
  n  10  11

If we want to join a dataframe with another that has a hierarchical index, we need to specify as many columns as there are levels in the index

In [49]:
pd.merge(left, right_h, left_on=['key1', 'key2'], right_index=True)

,key1,key2,lval,0,1
0,a,b,0,0,1
0,a,b,0,2,3


When we join _left_, _right_ or _outer_, there might be fields in the resulting dataframe that were not present in the inputs. These will be labeled with NaNs

In [50]:
pd.merge(left, right_h, left_on=['key1', 'key2'], right_index=True, how = 'outer')

,key1,key2,lval,0,1
0,a,b,0,0,1
0,a,b,0,2,3
1,a,n,1,NaN,NaN
2,b,m,2,NaN,NaN
2,b,n,NaN,4,5
2,b,n,NaN,6,7
2,x,n,NaN,8,9
2,x,n,NaN,10,11


In [51]:
pd.merge(left, right_h, left_on=['key1', 'key2'], right_index=True, how = 'left')

,key1,key2,lval,0,1
0,a,b,0,0,1
0,a,b,0,2,3
1,a,n,1,NaN,NaN
2,b,m,2,NaN,NaN


## Concatenating
Ie: joining side by side or top to bottom, without cross-referencing the indexes.

In [52]:
arr = np.arange(12).reshape(3,4)

In [57]:
np.arange(12)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [53]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [56]:
np.concatenate([arr,arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [58]:
s1 = pd.Series([0,1], index = ['a', 'b'])

s2 = pd.Series([2,3,4], index = list('cde'))

s3 = pd.Series([5,6], index = ['f', 'g'])

In [61]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [63]:
x = pd.concat([s1,s2,s3], axis=1)

In [64]:
type(x)

pandas.core.frame.DataFrame

In [65]:
x

,0,1,2
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [66]:
pd.concat([s1,s2,s3], axis=1, keys=list('ijk'))

,i,j,k
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


We can specify the names of the resulting columns

In [68]:
df6 = pd.concat([s1,s2,s3], axis=1, keys=list('ijk')).fillna(42)

In [69]:
df6

,i,j,k
a,0,42,42
b,1,42,42
c,42,2,42
d,42,3,42
e,42,4,42
f,42,42,5
g,42,42,6


In [70]:
df7 = pd.DataFrame(np.random.randn(2,2), columns = ['i','j'])

In [71]:
df7

,i,j
0,1.02483,0.928527
1,0.33691,-0.278200


In [72]:
pd.concat([df6,df7])

,i,j,k
a,0.00000,42.000000,42
b,1.00000,42.000000,42
c,42.00000,2.000000,42
d,42.00000,3.000000,42
e,42.00000,4.000000,42
f,42.00000,42.000000,5
g,42.00000,42.000000,6
0,1.02483,0.928527,NaN
1,0.33691,-0.278200,NaN


In [73]:
df6.append(df7)

,i,j,k
a,0.00000,42.000000,42
b,1.00000,42.000000,42
c,42.00000,2.000000,42
d,42.00000,3.000000,42
e,42.00000,4.000000,42
f,42.00000,42.000000,5
g,42.00000,42.000000,6
0,1.02483,0.928527,NaN
1,0.33691,-0.278200,NaN
